In [1]:
# ==== 1. Install ====
!pip -q install -U autogluon.tabular==1.4.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 8.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 66.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 12.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.7.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.2 which is incompatible.


In [2]:
# ==== 2. Load dataset ====
import pandas as pd
from autogluon.tabular import TabularPredictor

df = pd.read_csv("/kaggle/input/heart-disease-dataset/heart.csv")
label = "target"
print(df.shape)
df.head()


(1025, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [3]:
# ==== 3. Add a few manual features (for demo) ====
df["chol_to_age"] = df["chol"] / (df["age"] + 1)
df["bp_diff"] = df["trestbps"] - df["oldpeak"]
df["is_senior"] = (df["age"] > 60).astype(int)
print("Added feature columns:", [c for c in df.columns if c not in ["age","chol","trestbps","oldpeak","target"]])


Added feature columns: ['sex', 'cp', 'fbs', 'restecg', 'thalach', 'exang', 'slope', 'ca', 'thal', 'chol_to_age', 'bp_diff', 'is_senior']


In [4]:
# ==== 4. AutoGluon setup + Auto Feature Engineering ====
predictor = TabularPredictor(label=label, problem_type="binary", path="ag_heart_fe")

predictor.fit(
    train_data=df,
    time_limit=120,   # 2 minutes
    presets="medium_quality",
    verbosity=2,
)
leader = predictor.leaderboard(silent=True)
print(leader.head(10))


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       29.88 GB / 31.35 GB (95.3%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['medium_quality']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 120s
AutoGluon will save models to "/kaggle/working/ag_heart_fe"
Train Data Rows:    1025
Train Data Columns: 16
Label Column:       target
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    30598.58 MB
	Train Data (Original)  Memory Usage: 0.13 MB (0.0% of available memory)
	Inferring 

                 model  score_val eval_metric  pred_time_val   fit_time  \
0        LightGBMLarge        1.0    accuracy       0.001347   1.093103   
1             LightGBM        1.0    accuracy       0.001791   0.504736   
2           LightGBMXT        1.0    accuracy       0.003227  10.682480   
3             CatBoost        1.0    accuracy       0.003500   1.311315   
4              XGBoost        1.0    accuracy       0.003799   0.271341   
5  WeightedEnsemble_L2        1.0    accuracy       0.004756   0.353500   
6       NeuralNetTorch        1.0    accuracy       0.008717  13.735564   
7     RandomForestEntr        1.0    accuracy       0.088199   0.971641   
8     RandomForestGini        1.0    accuracy       0.099861   1.056627   
9       ExtraTreesEntr        1.0    accuracy       0.101648   1.019702   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                0.001347           1.093103            1       True   
1                0.001791     

In [7]:
from autogluon.features import AutoMLPipelineFeatureGenerator

X = df.drop(columns=[label])
gen = AutoMLPipelineFeatureGenerator()
_ = gen.fit_transform(X)

# Version-robust access
fm = getattr(gen, "feature_metadata_out", None) or getattr(gen, "feature_metadata_out_", None)
if fm is not None:
    try:
        print(fm.type_map_raw)
    except AttributeError:
        print(fm.get_type_map_raw())
else:
    # Fallback: just show dtypes of engineered features
    print("No feature_metadata_out on this version; showing dtypes instead:")
    print(_.dtypes.value_counts())


Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    30571.75 MB
	Train Data (Original)  Memory Usage: 0.13 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 4 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) :  3 | ['oldpeak', 'chol_to_age', 'bp_diff']
		('int', [])   : 13 | ['age', 'sex', 'cp', 'trestbps', 'chol', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 3 | ['oldpeak', 'chol_to_age',

No feature_metadata_out on this version; showing dtypes instead:
int64      9
int8       4
float64    3
Name: count, dtype: int64
